In [140]:
import os
import sys
from tweepy import API
from tweepy import OAuthHandler
from tweepy import Cursor
import json
from collections import Counter
import math
import time
import pandas as pd
import csv
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import time
import threading

In [141]:
#fill the spaces with your twitter developer credentials
def getTwitterOAuth():
    consumer_key = ''
    consumer_secret = ''
    access_token = ''
    access_token_secret = ''
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token,access_token_secret)
    return auth

def TwitterClient():
    auth = getTwitterOAuth()
    client = API(auth)
    return client
#global variables
MAX_FRIENDS = 15000
max_pages = math.ceil(MAX_FRIENDS/5000)
        

In [142]:
'''
def makeDirectory():
    client = TwitterClient()
    #dirname = "users/{}".format(user)
    dirname = "users"
    
    try:
        os.makedirs(dirname, mode =0o755, exist_ok=True)
    except OSError:
        print("Directory {} already exist".format(dirname))
    except Exception as e:
        print( "Error while creating directory {}".format(dirname))
        print(e)
        sys.exit(1)
'''


In [143]:
def paginate(items,n):
    for i in range(0, len(items),n):
        yield items[i:i+n] 

In [ ]:
usrSet = set()
usrSet2 = set()


In [210]:
# user friends
usrset3 = set()

def userFriends(user="AvokaAyebilla"):
    client = TwitterClient()
    fname = "{}_friends.jsonl".format(user)
    with open(fname, 'w') as f:
        for friends in Cursor(client.friends_ids,screen_name=user).pages(max_pages):
            for chunk in paginate(friends, 100):
                users = client.lookup_users(user_ids=chunk)
                for usr in users:
                    if usr.location =='Ghana':
                        usrset3.add(usr.screen_name)
                        f.write(json.dumps(usr._json)+"\n")
            if len(friends) == 100:
                print("More results available. Sleeping for 1 minute to avoid rate limit..")
                time.sleep(120)
    #return usrSet2


Following are list of some identified top Ghanaians in different industries 
identified. Their twitter handles were obtained by searching their names on twitter
We then get their friends and add it to form the top 1000 Ghanaians on twitter.
Repeaton was disallowed by using the python set.


In [211]:
useList = ['JDMahama','NAkufoAddo','Manasseh_Azure','papakwesijunior','shattawalegh','ASAMOAH_GYAN3','PatrickAwuahJr','MBawumia',
              'SBawumia','anasglobal','officeofJJR','nm_appiah','konkrumah','AmgMedikal','RAkufoAddo',
              'NJOAgyemang','S_OkudzetoAblak','MensaOtabil','ArchbishopNick']

list2 = ['S_OkudzetoAblak','MensaOtabil','ArchbishopNick','MBawumia','SBawumia','johndumelo1','Serwaa_Amihere',
        'MajidMichelMM','vanvicker1','sarkodie','samini_dagaati','CharlotteAmaOs1','nanaotafrija','akumaamamazimbi','KafuiDey','Jackie Appiah',
        'CoachOpeele','Citi973','TheGhanaWeb']

list3 = ['NJOAgyemang','S_OkudzetoAblak','MensaOtabil','ArchbishopNick']

'''
this is a call of the userFriends function. you will have to manually change the name at  usrset3.add(usr.screen_name)
I did this because i wanted to divide my input into smaller chunks for faster processing. 
Each call of this method is given a separate list of users as input

 So simply provide the set name in the userFrinds function and the list that contains the input here in the forloop
 The output of this is a set containing the friends of the users you passed as an input
''' 


for i in list3:
    try:
        userFriends(i)
    except Exception as e:
        print( "Error while getting friends of  {}".format(i))
        print(e)


In [212]:
#Turn write the output into a csv file for persistence. Use different file names for every call so you don't override the previous

df = pd.DataFrame(usrSet, columns = ['handles'])
#df
df.to_csv('Top_ghanaians3.csv', sep=',', index =False, encoding='utf-8')


In [158]:
#different csv file for second call to the method
df = pd.DataFrame(usrSet2)
df
df.to_csv('Top_ghanaians2.csv', sep=',', index =False, encoding='utf-8')


In [222]:
#Now load the different csv files into separate sets then unite them to form the final list of user friends
handset1 = set()
handset2 = set()
handset3 = set()

In [227]:
#load a csv file at time. I had to first load the data from the csv into a list before converting it to a set
#This was manual so be careful
handles = []
def readfile():
    with open('Top_ghanaians.csv', 'rt') as f:
        f.readline()
        csv_reader = csv.reader(f)
        for line in csv_reader:
            handles.append(line)         
readfile()           

In [228]:
#Here, list is converted into a set
for i in handles:
    for j in i:
        handset1.add(j)
#handset2




In [246]:
#remember the individual list you used as input to find the user friends were not written to the csv file
# Concatenate that list with the list you loaded from the csv file to form one list of all your twitter users
handset4 = set()
userList = useList + list3 + list2
for i in userList:
    handset4.add(i)


In [248]:
#use the union method of set to unite the different sets of users you have made.
usrhandles = set.union(handset1, handset2, handset3,handset4)

In [250]:
#write the result back to a csv file. Now you have a complete list of users for your country.
df = pd.DataFrame(usrhandles, columns = ['user_handles'])
df.to_csv('Top_ghanaians_usrs.csv', sep=',', index =False, encoding='utf-8')